In [3]:
from tabulate import tabulate
from copy import deepcopy
f = open("input.txt", "r")

rows = []

for line in f:
    [machines, groups] = line.replace('\n', '').split(' ')
    groups = tuple([int(c) for c in groups.split(',')])
    rows.append({"machines": list(machines), "groups": groups})

print(tabulate(rows))

def computeGroupsNoUnknown(machines):
    groups = []
    index = 0
    while index < len(machines) :
        c = machines[index]
        index += 1
        nb_in_group = 0
        while c == '#' :
            nb_in_group += 1
            if index < len(machines):
                c = machines[index]
                index += 1
            else:
                break
        if nb_in_group > 0:
            groups.append(nb_in_group)
    return tuple(groups)

def computeGroups(machines):
    try:
        first_unknown = machines.index('?')
        if_damaged = deepcopy(machines)
        if_op = deepcopy(machines)
        if_damaged[first_unknown] = '#'
        groups_if_damaged = computeGroups(if_damaged)
        if_op[first_unknown] = '.'
        groups_if_op = computeGroups(if_op)
        return groups_if_damaged + groups_if_op
    except ValueError:
        return [computeGroupsNoUnknown(machines)]

total = 0

for row in rows:
    machines = row["machines"]
    actual_groups = row["groups"]
    print(machines)
    groups = computeGroups(row["machines"])
    count = groups.count(actual_groups)
    print(count)
    total += count

print(total)


----------------------------------------------------------------------------------------------------  ------------------
['?', '?', '?', '?', '?', '?', '#', '?', '?', '#', '?', '?']                                          (1, 1, 5, 1)
['?', '#', '?', '#', '?', '?', '#', '#', '?', '#', '?']                                               (2, 5, 1)
['?', '?', '?', '?', '?', '?', '?', '?', '.', '?', '#', '?', '?', '?', '#', '?', '?', '#', '#', '?']  (2, 1, 2, 1, 1, 6)
['#', '?', '?', '?', '#', '?', '?', '?', '?', '?', '.', '?', '#', '?', '.']                           (2, 1, 2, 1)
['?', '#', '#', '.', '.', '?', '#', '?', '#', '?', '?']                                               (2, 4)
['.', '.', '#', '?', '?', '#', '#', '?', '?', '?', '?', '#', '#', '#', '?', '?', '?', '?', '#', '?']  (6, 8)
['?', '?', '?', '#', '.', '.', '?', '#', '?', '?']                                                    (1, 1, 2)
['.', '#', '?', '?', '?', '?', '#', '?', '.', '?', '?', '?']                          

In [20]:
from tabulate import tabulate
from collections import deque
import math as m
from copy import deepcopy

f = open("input.txt", "r")

rows = []

for line in f:
    [machines, groups] = line.replace('\n', '').split(' ')
    original_groups = [int(c) for c in groups.split(',')]
    unfolded_groups = []
    unfolded_groups += original_groups
    unfolded_machines = "" + machines
    for i in range(4):
        unfolded_groups += original_groups
        unfolded_machines += '?' + machines
    rows.append({"machines": unfolded_machines, "original_groups": original_groups, "unfolded_groups": unfolded_groups, "arrangements": {}})

print(tabulate(rows))

def buildStringFromList(string_list):
    result = ""
    for s in string_list:
        result += s
    return result

def checkCompatibility(s, valid_s):
    if len(s) != len(valid_s):
        return False
    for i in range(len(s)):
        c0 = s[i]
        c1 = valid_s[i]
        if c0 == c1:
            continue
        match (c0, c1):
            case ('.', '#') | ('#', '.'):
                return False
            case _:
                continue
    return True

def computeArrangements(machines, original_groups, unfolded_groups, starting_index, length):
    # Build the shortest string that matches the group requirements
    # Split it into a list of '.' groups and '#' groups
    min_string_length = 0
    min_string_list = []
    min_string_list.append("")
    for group in unfolded_groups[starting_index:starting_index+length]:
        min_string_length += group
        group_string = ""
        for i in range(group):
            group_string += '#'
        min_string_list.append(group_string)
        min_string_list.append('.')
        min_string_length += 1
    min_string_list[-1] = ""
    # At this point, min_string_list contains a list of '.' and '#' groups, as strings.
    # The first and last groups are empty, since no dot is required there to match the constraints
    # For example: ["", "###", ".", "##", ".", "####", ""]
    min_string_length = max(0, min_string_length-1)
    # Check if the length of the machines string is enough to match the constraints
    # (Should always be the case)
    if len(machines) < min_string_length:
        return 0

    # If the machines string has exactly the same length as the min string, then the min string
    # is the only possible matching string so we just check if it's compatible with the machines string
    if min_string_length == len(machines):
        if checkCompatibility(machines, buildStringFromList(min_string_list)):
            return 1
        else:
            return 0

    possibilities = [{} for i in range(length+1)]
    # We can figure out how many '.' characters need to be added to the min string to get
    # a possible string
    max_nb_of_added_dots = len(machines) - min_string_length

    # For each '.' group in min string, we'll add '.' characters then check if the machines substring
    # and the string composed of the resulting '.' group and the next '#' groups match.
    # Keep track of the nb of possibilities for each '.' sub group, by length of resulting string
    # For example, with ["", "###", ".", "##", ".", "####", ""]:
    # Check for ###, .###, ..###, etc. against machines[0:3], machines[0:4], machines [0:5], etc
    # and store in possibilities, for subgroup 0: {3: 1, 4: 1, 5: 1} if they're all compatible
    # then check 
    # .##, ..##, ...##, etc. against machines[3:x],
    # .##, ..##, ...##, etc. against machines[4:y],
    # .##, ..##, ...##, etc. against machines[5:z]
    # and store the resulting possibilities for subgroup 1: {x: n*1, y: m*1, z: p*1}
    # And so on until all groups have been used
    for subgroup_index in range(length+1):
        if subgroup_index == 0:
            # There is one possibility for a string of length 0 at this point
            prev_possibilities = {0: 1}
        else:
            prev_possibilities = possibilities[subgroup_index-1]
        for prev_length, prev_poss in prev_possibilities.items():
            # We loop through the possible string lengths up to now
            s0 = min_string_list[subgroup_index*2]
            if subgroup_index*2+1 >= len(min_string_list):
                s1 = ""
            else:
                s1 = min_string_list[subgroup_index*2+1]
            if len(s0) + len(s1) > len(machines):
                continue
            string_to_check = buildStringFromList([s0, s1])
            for i in range(max_nb_of_added_dots+1):
                if len(string_to_check) > len(machines):
                    # No need to go on if we've exceeded the max length
                    break
                if checkCompatibility(machines[prev_length:prev_length+len(string_to_check)], string_to_check):
                    # We found a string matching the portion of machines starting at prev_length
                    if prev_length+len(string_to_check) in possibilities[subgroup_index]:
                        possibilities[subgroup_index][prev_length+len(string_to_check)] += prev_poss
                    else:
                        possibilities[subgroup_index][prev_length+len(string_to_check)] = prev_poss
                string_to_check = "." + string_to_check

    if len(machines) in possibilities[-1]:
        result = possibilities[-1][len(machines)]
    else:
        result = 0

    return result

total = 0

for row_nb, row in enumerate(rows):
    print(f'Row {row_nb} of {len(rows)-1}')
    machines = row["machines"]
    unfolded_groups = row["unfolded_groups"]
    original_groups = row["original_groups"]
    print(machines, unfolded_groups)
    total += computeArrangements(machines, original_groups, unfolded_groups, 0, len(unfolded_groups))
    print(f"Total after for row {row_nb}: {total}")
print(total)
    



-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ---------------------------------------------------------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  --
?????#??????????#??????#??????????#??????#??????????#??????#??????????#??????#??????????#   